In [11]:
%matplotlib inline
import warnings
import itertools
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
from datetime import date
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
import matplotlib.pyplot as plt
from datetime import date
from dateutil.relativedelta import relativedelta
import sys
from IPython.core.debugger import set_trace


###Load Data

In [12]:
m3_year = pd.read_excel('../data/M3C.xls',0)
m3_quar = pd.read_excel('../data/M3C.xls',1)
m3_mont = pd.read_excel('../data/M3C.xls',2)
m3_mont = m3_mont[m3_mont['Starting Year'] != 0]

len(m3_year)+len(m3_quar)+len(m3_mont)
print(len(m3_quar))

756


In [13]:
def convert_to_date(init_year, init_quarter_month, x, quarter_multiplier):
    year = int(init_year)
    new_date = date(year,1,1) + relativedelta(months=+quarter_multiplier*init_quarter_month) + relativedelta(months=+quarter_multiplier*x)
    return new_date

In [59]:
def series_to_dataframes(values, dates):
    size = int(len(values) * 0.3)
    train, test = values[0:size], values[size:len(values)]
    train_dates, test_dates = dates[0:size], dates[size:len(values)]
    
    train_df = pd.DataFrame(data= train, index = train_dates, columns = ["value"])
    train_df = train_df.convert_objects(convert_numeric=True)
    train_df = train_df[np.isfinite(train_df['value'])]
    
    test_df = pd.DataFrame(data= test, index = test_dates, columns = ["value"])
    test_df = test_df.convert_objects(convert_numeric=True)
    test_df = test_df[np.isfinite(test_df['value'])]
    
    print(test_df.values)

    return train_df, test_df

###Transform to right format as a list of pandas dataframes each containing a timeseries (

In [60]:
train_year_dfs = []
test_year_dfs = []

for i in range (0,len(m3_year)):
    series = m3_year.iloc[i,6:53]
    init_year = m3_year.iloc[i,4]
    dates_list = [date(int(init_year+x),1,1) for x in range(0,len(series))] 
    
    train_df, test_df = series_to_dataframes(series.values, dates_list)
    train_year_dfs.append(train_df)
    test_year_dfs.append(test_df)    
             
train_quar_dfs = []        
test_quar_dfs = []
for i in range (0,len(m3_quar)):
    series = m3_quar.iloc[i,6:77]
    init_year = m3_quar.iloc[i,4]
    init_quar = m3_quar.iloc[i,5]
    dates_list =  [convert_to_date(init_year,init_quar,x, 3) for x in range(0,len(series))] 
    
    train_df, test_df = series_to_dataframes(series.values, dates_list)
    train_quar_dfs.append(train_df)
    test_quar_dfs.append(test_df)
    
train_mont_dfs = []
test_mont_dfs = []
for i in range (0,len(m3_mont)):
    series = m3_mont.iloc[i,6:149]
    init_year = m3_mont.iloc[i,4]
    init_month = m3_mont.iloc[i,5]
    dates_list =  [convert_to_date(init_year,init_month,x, 1) for x in range(0,len(series))] 
    
    train_df, test_df = series_to_dataframes(series.values, dates_list)
    train_mont_dfs.append(train_df)
    test_mont_dfs.append(test_df)

[[5379.75]
 [6158.68]
 [6876.58]
 [7851.91]
 [8407.84]
 [9156.01]]
[[4793.2]
 [5602. ]
 [5065. ]
 [5056. ]
 [5067.2]
 [5209.6]]
[[3070.2]
 [3601.6]
 [3407.4]
 [3500.6]
 [3437.8]
 [3007. ]]
[[4656.  ]
 [5228.52]
 [5656.72]
 [5077.02]
 [5403.4 ]
 [5009.52]]
[[5250.9]
 [4899.2]
 [4317.9]
 [4007.9]
 [4323.4]
 [4819.4]]
[[4338.03]
 [4185.92]
 [3950.49]
 [3715.59]
 [3958.3 ]
 [4140.39]]
[[5163.21]
 [5089.94]
 [4786.4 ]
 [4926.47]
 [5068.83]
 [5318.19]]
[[8809.8 ]
 [8953.5 ]
 [7991.01]
 [7294.08]
 [7163.37]
 [7478.31]]
[[6277.6 ]
 [7446.28]
 [6741.69]
 [6600.69]
 [7850.17]
 [8485.26]]
[[10042.9]
 [10173. ]
 [ 8751. ]
 [ 3472. ]
 [ 3187. ]
 [ 3058. ]]
[[5388.4]
 [5766.2]
 [5937.9]
 [5627.4]
 [5982.7]
 [5931.6]]
[[ 5949.5]
 [ 6616.5]
 [ 7247. ]
 [ 8205. ]
 [10158.5]
 [12495.5]]
[[6271. ]
 [6901.8]
 [6479.2]
 [6452.3]
 [6271.6]
 [6405.2]]
[[4010.8]
 [4120.8]
 [4084.2]
 [4330.2]
 [4552.4]
 [4724. ]]
[[5995. ]
 [6510.5]
 [6670. ]
 [6941.5]
 [7010. ]
 [7539.5]]
[[6658.3]
 [7101.4]
 [7375.8]
 [5748.

/home/cartest/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  import sys
/home/cartest/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # This is added back by InteractiveShellApp.init_path()



[[3676.83]
 [4721.37]
 [5243.28]
 [6276.36]
 [7026.27]
 [8885.16]]
[[12523.5]
 [12166. ]
 [12881.5]
 [13480.5]
 [ 7086.5]
 [ 7598. ]]
[[4227.]
 [4481.]
 [4395.]
 [4602.]
 [4829.]
 [4841.]]
[[6747.02]
 [6775.18]
 [7079.44]
 [7873.68]
 [8304.85]
 [8966.21]]
[[ 7928.83]
 [ 7751.34]
 [ 8468.2 ]
 [ 9231.5 ]
 [ 9811.36]
 [11374.3 ]]
[[5025. ]
 [5062.6]
 [4878.6]
 [5099.6]
 [5050.8]
 [5505.4]]
[[4421.52]
 [4417.08]
 [3819.84]
 [3816.98]
 [3472.72]
 [4063.7 ]]
[[8061.94]
 [8551.94]
 [9303.02]
 [9592.14]
 [9611.4 ]
 [9505.  ]]
[[3070.2]
 [3601.6]
 [3407.4]
 [3500.6]
 [3437.8]
 [3007. ]]
[[ 3274.64]
 [ 3970.12]
 [ 5589.92]
 [ 9170.12]
 [ 8793.24]
 [10438.96]]
[[2509.23]
 [2835.65]
 [3193.03]
 [3756.92]
 [5400.23]
 [8299.04]]
[[ 8388.45]
 [ 8570.2 ]
 [ 8858.75]
 [ 9702.85]
 [11116.85]
 [12344.85]]
[[3464.8]
 [5180.4]
 [5090.2]
 [5245.8]
 [5217.4]
 [5713.4]]
[[2825.79]
 [2994.88]
 [3546.68]
 [5304.05]
 [6379.21]
 [7146.98]]
[[6462.07]
 [5119.36]
 [4035.35]
 [4815.54]
 [7010.89]
 [4328.  ]]
[[3889

 [12577.6]]
[[  656.85]
 [  695.2 ]
 [  760.9 ]
 [ 1019.25]
 [ 1377.35]
 [ 1546.8 ]
 [ 1537.45]
 [ 1414.85]
 [ 1235.25]
 [ 1228.95]
 [ 1346.1 ]
 [ 1483.05]
 [ 1775.8 ]
 [ 1751.9 ]
 [ 1985.1 ]
 [ 2265.45]
 [ 2858.  ]
 [ 3927.4 ]
 [ 4486.6 ]
 [ 4735.5 ]
 [ 5419.55]
 [ 5829.25]
 [ 6625.35]
 [ 7147.65]
 [ 7410.6 ]
 [ 7932.5 ]
 [ 9575.9 ]
 [12620.05]
 [12503.95]
 [12379.85]
 [11285.95]]
[[ 1057.2]
 [ 1095. ]
 [ 1165.8]
 [ 1369.6]
 [ 1571.1]
 [ 1795. ]
 [ 1634.3]
 [ 1429.8]
 [ 1288.2]
 [ 1399.7]
 [ 1532.2]
 [ 1669.5]
 [ 2088.9]
 [ 2216.8]
 [ 2414.1]
 [ 2611.9]
 [ 3022.3]
 [ 3037.7]
 [ 3719.9]
 [ 4659.1]
 [ 6320.1]
 [ 7443.5]
 [ 8562.6]
 [ 9233.4]
 [ 9500.9]
 [ 9243.9]
 [ 9239.4]
 [10779.7]
 [ 8201.7]
 [ 8482.7]
 [ 8124.1]]
[[ 774.45]
 [ 769.45]
 [ 866.05]
 [ 985.45]
 [1413.35]
 [2072.1 ]
 [2298.  ]
 [2165.45]
 [2380.1 ]
 [2373.4 ]
 [2829.35]
 [3127.65]
 [3101.6 ]
 [3224.7 ]
 [4393.25]
 [6599.8 ]
 [7319.1 ]
 [7377.55]
 [6543.4 ]]
[[ 1951. ]
 [ 1751.5]
 [ 1779. ]
 [ 2677. ]
 [ 2801. ]
 [ 4614.

[[4856. ]
 [5000. ]
 [5185. ]
 [5401.5]
 [5626.5]
 [5826. ]
 [5970. ]
 [6109. ]
 [6219.5]]
[[4836.5]
 [5000. ]
 [5169. ]
 [5387.5]
 [5670. ]
 [5913.5]
 [6146. ]
 [6282. ]
 [6390. ]]
[[4884.5]
 [5000. ]
 [5159. ]
 [5325.5]
 [5370.5]
 [5558. ]
 [5785.5]
 [5973.5]
 [6072. ]]
[[4552.5]
 [5000. ]
 [5284. ]
 [5231. ]
 [5089.5]
 [4944.5]
 [5086. ]
 [5070.5]
 [5144.5]]
[[4869. ]
 [5000. ]
 [5243. ]
 [5496. ]
 [5639.5]
 [5717. ]
 [5709. ]
 [5752.5]
 [5645. ]]
[[4929.5]
 [5000. ]
 [5141. ]
 [5247.5]
 [5360. ]
 [5479.5]
 [5561.5]
 [5643. ]
 [5626.5]]
[[4817.5]
 [5000. ]
 [5321. ]
 [5614. ]
 [6031. ]
 [6232.5]
 [6274. ]
 [6145. ]
 [6156. ]]
[[4791. ]
 [5000. ]
 [5180. ]
 [5325. ]
 [5517.5]
 [5623.5]
 [5688.5]
 [5654. ]
 [5782. ]]
[[4744. ]
 [5000. ]
 [5220. ]
 [5326. ]
 [5506.5]
 [5688. ]
 [5932. ]
 [6171. ]
 [6341.5]]
[[4904.5]
 [5000. ]
 [5083. ]
 [5103.5]
 [5120. ]
 [5160.5]
 [5222. ]
 [5359.5]
 [5472. ]]
[[4847. ]
 [5000. ]
 [5078.5]
 [5166. ]
 [5379. ]
 [5577. ]
 [5741.5]
 [5849. ]
 [5944.5]]

...

 [6485.9 ]]
[[5591.6 ]
 [5575.75]
 [5511.45]
 [5542.85]
 [5537.55]
 [5529.4 ]
 [5454.2 ]
 [5473.2 ]
 [5511.45]
 [5552.05]
 [5618.85]
 [5674.9 ]
 [5732.6 ]
 [5726.35]
 [5651.35]
 [5694.9 ]
 [5690.45]
 [5683.95]
 [5606.95]
 [5623.  ]
 [5643.35]
 [5695.25]
 [5711.1 ]
 [5810.45]
 [5853.3 ]
 [5836.85]
 [5773.7 ]
 [5812.5 ]
 [5815.7 ]
 [5798.9 ]
 [5739.3 ]
 [5751.15]
 [5792.2 ]
 [5817.35]
 [5851.95]
 [5935.95]
 [5975.1 ]
 [5954.6 ]
 [5874.9 ]
 [5909.7 ]
 [5908.4 ]
 [5884.9 ]
 [5801.85]
 [5815.4 ]
 [5854.65]
 [5867.95]
 [5913.85]
 [5980.25]
 [5997.7 ]
 [5958.7 ]
 [5898.05]
 [5914.95]
 [5880.55]
 [5864.35]
 [5749.5 ]
 [5757.55]
 [5781.95]
 [5833.9 ]
 [5831.2 ]
 [5914.  ]
 [5937.55]
 [5892.95]
 [5866.75]
 [5877.75]
 [5855.5 ]
 [5827.45]
 [5756.1 ]
 [5761.2 ]
 [5805.3 ]
 [5846.65]
 [5876.75]
 [5945.35]
 [5987.7 ]
 [5954.1 ]
 [5897.65]
 [5912.3 ]
 [5911.95]
 [5903.65]
 [5806.15]
 [5836.75]
 [5870.3 ]
 [5892.8 ]
 [5960.05]
 [6029.3 ]
 [6066.15]
 [6050.1 ]
 [5986.5 ]
 [6022.3 ]
 [6031.6 ]
 [6031.8 

[[4600.]
 [4610.]
 [4610.]
 [4605.]
 [4585.]
 [4625.]
 [4585.]
 [4570.]
 [4565.]
 [4585.]
 [4585.]
 [4610.]
 [4675.]
 [4600.]
 [4630.]
 [4640.]
 [4640.]
 [4625.]
 [4620.]
 [4610.]
 [4615.]
 [4595.]
 [4590.]
 [4570.]
 [4545.]
 [4520.]
 [4515.]
 [4510.]
 [4525.]
 [4525.]
 [4540.]
 [4560.]
 [4575.]
 [4560.]
 [4555.]
 [4540.]
 [4525.]
 [4530.]
 [4520.]
 [4505.]
 [4500.]
 [4500.]
 [4480.]
 [4450.]
 [4410.]
 [4400.]
 [4375.]
 [4360.]
 [4305.]
 [4315.]
 [4315.]
 [4280.]
 [4225.]
 [4195.]
 [4175.]
 [4175.]
 [4175.]
 [4145.]
 [4165.]
 [4180.]
 [4210.]
 [4215.]
 [4230.]
 [4250.]
 [4275.]
 [4260.]
 [4245.]
 [4240.]
 [4235.]
 [4235.]
 [4240.]
 [4225.]
 [4255.]
 [4195.]
 [4175.]
 [4165.]
 [4160.]
 [4160.]
 [4165.]
 [4160.]
 [4170.]
 [4140.]
 [4145.]
 [4115.]
 [4095.]
 [4065.]
 [4060.]
 [4025.]
 [4025.]
 [3985.]
 [3975.]
 [3970.]
 [3955.]]
[[6320.]
 [6370.]
 [6410.]
 [6430.]
 [6440.]
 [6450.]
 [6480.]
 [6500.]
 [6520.]
 [6530.]
 [6520.]
 [6480.]
 [6500.]
 [6510.]
 [6520.]
 [6560.]
 [6580.]
 [6620.]


[[6033. ]
 [6095.6]
 [6074.4]
 [6108. ]
 [6273.2]
 [6190.4]
 [6268. ]
 [6377.2]
 [6330.8]
 [6330.8]
 [6330.8]
 [6282.6]
 [6248.2]
 [6362.8]
 [6285.6]
 [6342.2]
 [6454.2]
 [6343.2]
 [6440.2]
 [6519.4]
 [6368.8]
 [6501.6]
 [6454.6]
 [6401. ]
 [6490.8]
 [6408. ]
 [6497.4]
 [6648.6]
 [6578.8]
 [6651.6]
 [6815.6]
 [6747.4]
 [6711.6]
 [6921.4]
 [6685.6]
 [6766.2]
 [6921.4]
 [6780.6]
 [6828. ]
 [6996.2]
 [6878. ]
 [6920.6]
 [6970.4]
 [7040. ]
 [6999. ]
 [7184. ]
 [7041.6]
 [7081.2]
 [7232.8]
 [7090. ]
 [7285. ]
 [7149.8]
 [7198.4]
 [7399.2]
 [7283.8]
 [7360.4]
 [7340.8]
 [7362. ]
 [7341.4]
 [7522.4]
 [7341.4]
 [7382.8]
 [7560. ]
 [7426.6]
 [7490.2]
 [7696.6]
 [7582.8]
 [7862.4]
 [7804.8]
 [7668.4]
 [7654. ]
 [7611.2]
 [7632.6]
 [7862.6]
 [7696.4]
 [7760.8]
 [8000.2]
 [7853.4]
 [7946.8]
 [7989.6]
 [7930.8]
 [8004. ]
 [8210.2]
 [7974.4]
 [8002.8]
 [8262.8]
 [8102.6]
 [8196.8]
 [8202.2]
 [8282.4]
 [8523.2]
 [8384.4]
 [8332.4]]
[[5287.4]
 [5287.4]
 [5320. ]
 [5339.6]
 [5421. ]
 [5397.8]
 [5420. ]

[[3100.]
 [3300.]
 [2900.]
 [3100.]
 [3200.]
 [3300.]
 [3400.]
 [3300.]
 [3300.]
 [3400.]
 [3000.]
 [2000.]
 [2600.]
 [3100.]
 [3600.]
 [4700.]
 [5000.]
 [5500.]
 [5800.]
 [6200.]
 [6600.]
 [7400.]
 [7900.]
 [8400.]
 [8400.]
 [7900.]
 [8800.]
 [8600.]
 [9200.]
 [9000.]]
[[1290.]
 [ 990.]
 [1330.]
 [1250.]
 [1480.]
 [1460.]
 [1400.]
 [1640.]
 [1370.]
 [1280.]
 [1560.]
 [1350.]
 [1390.]
 [1070.]
 [1590.]
 [1250.]
 [1660.]
 [1640.]
 [1730.]
 [1740.]
 [1590.]
 [1100.]
 [1230.]
 [1380.]
 [ 950.]
 [ 860.]
 [1060.]
 [ 930.]
 [ 780.]]
[[ 710.]
 [ 660.]
 [ 830.]
 [ 830.]
 [1500.]
 [1930.]
 [1620.]
 [2550.]
 [1730.]
 [1680.]
 [2010.]
 [1270.]
 [1670.]
 [1170.]
 [4220.]
 [1920.]
 [2510.]
 [1910.]
 [2580.]
 [2300.]
 [2200.]
 [1490.]
 [1690.]
 [2220.]
 [1470.]
 [1230.]
 [2740.]
 [2040.]
 [2110.]]
[[3679.]
 [3499.]
 [4289.]
 [5267.]
 [4305.]
 [5484.]
 [4252.]
 [5740.]
 [4825.]
 [5127.]
 [4242.]
 [4826.]
 [3287.]
 [4675.]
 [5293.]
 [4933.]
 [4598.]
 [5576.]
 [4661.]
 [4579.]
 [5617.]
 [3948.]
 [4209.

 [ 3680.]]
[[ 9280.]
 [10680.]
 [ 8840.]
 [ 6280.]
 [ 4800.]
 [ 4120.]
 [ 2720.]
 [ 3720.]
 [ 5480.]
 [ 6400.]
 [ 7720.]
 [ 8600.]
 [10080.]
 [10200.]
 [ 8800.]
 [ 6800.]
 [ 4000.]
 [ 1160.]
 [ 3400.]
 [ 3160.]
 [ 4480.]
 [ 6840.]
 [ 7840.]
 [ 9360.]
 [10440.]
 [ 9240.]
 [ 9160.]
 [ 6960.]
 [ 4320.]
 [ 5240.]
 [ 3280.]
 [ 4200.]
 [ 4600.]
 [ 6160.]
 [ 7080.]
 [ 9320.]
 [10080.]
 [ 9520.]
 [ 8320.]
 [ 6280.]
 [ 4960.]
 [ 3520.]
 [ 4600.]
 [ 3560.]
 [ 5360.]
 [ 5320.]
 [ 7960.]
 [ 9000.]
 [ 8880.]
 [ 9320.]
 [ 8360.]
 [ 5720.]
 [ 4680.]
 [ 3600.]]
[[ 8800.]
 [ 8960.]
 [ 8160.]
 [ 6120.]
 [ 5120.]
 [ 2480.]
 [ 2000.]
 [ 3640.]
 [ 5120.]
 [ 5480.]
 [ 6520.]
 [ 9240.]
 [10000.]
 [ 8720.]
 [ 8440.]
 [ 6520.]
 [ 4600.]
 [ 3720.]
 [ 1560.]
 [ 1240.]
 [ 5280.]
 [ 6880.]
 [ 7640.]
 [ 8880.]
 [ 9200.]
 [ 9640.]
 [ 8480.]
 [ 7160.]
 [ 4120.]
 [ 4000.]
 [ 4240.]
 [ 4000.]
 [ 5280.]
 [ 7200.]
 [ 8080.]
 [ 8680.]
 [10080.]
 [ 9520.]
 [ 8080.]
 [ 6840.]
 [ 4320.]
 [ 3080.]
 [ 4120.]
 [ 2800.]
 [ 4000.

In [61]:
len(m3_year)

645

In [ ]:
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

print(pdq)

[(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]


In [ ]:
# Brute force exploration of the best set of pdq parameters

def explore_pdq(series, pdq):
    results_aic = [sys.maxsize] * len(series)
    results = [None] * len(series)
    best_pdq = [None] * len(series)
    i = 0
    for ts in series:
        for param in pdq:
            try:
                model = ARIMA(ts, order=param)
                result = model.fit(disp=0)
                if result.aic < results_aic[i]:
                    results_aic[i] = result.aic
                    results[i] = result
                    best_pdq[i] = param
            except Exception as e:
                print(e)
                continue

        i = i + 1
    return results, best_pdq

train_year_results, best_year_pdq = explore_pdq(train_year_dfs,pdq)
train_month_results, best_month_pdq = explore_pdq(train_mont_dfs,pdq)
train_quart_results, best_quart_pdq = explore_pdq(train_quar_dfs,pdq)

The computed initial AR coefficients are not stationary
You should induce stationarity, choose a different model order, or you can
pass your own start_params.


/home/cartest/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/kalmanf/kalmanfilter.py:646: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if issubdtype(paramsdtype, float):
/home/cartest/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/kalmanf/kalmanfilter.py:650: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  elif issubdtype(paramsdtype, complex):
/home/cartest/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial AR coefficients are not stationary
You should induce stationarity, choose a different model order, or you can
pass your own start_params.


/home/cartest/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/tsatools.py:654: RuntimeWarning: divide by zero encountered in log
  invmacoefs = -np.log((1-macoefs)/(1+macoefs))
/home/cartest/anaconda3/lib/python3.6/site-packages/statsmodels/tools/numdiff.py:243: RuntimeWarning: invalid value encountered in subtract
  **kwargs)).imag/2./hess[i, j]
/home/cartest/anaconda3/lib/python3.6/site-packages/statsmodels/tools/numdiff.py:243: RuntimeWarning: invalid value encountered in multiply
  **kwargs)).imag/2./hess[i, j]
/home/cartest/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/tsatools.py:628: RuntimeWarning: invalid value encountered in exp
  newparams = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
/home/cartest/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/tsatools.py:629: RuntimeWarning: invalid value encountered in exp
  tmp = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
/home/cartest/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: Co

The computed initial AR coefficients are not stationary
You should induce stationarity, choose a different model order, or you can
pass your own start_params.
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial AR coefficients are not stationary
You should induce stationarity, choose a different model order, or you can
pass your own start_params.


...

The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.


/home/cartest/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial AR coefficients are not stationary
You should induce stationarity, choose a different model order, or you can
pass your own start_params.
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial MA coefficients are 

/home/cartest/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial AR coefficients are not stationary
You should induce stationarity, choose a different model order, or you can
pass your own start_params.
The computed initial AR coefficients are not stationary
You should induce stationarity, choose a different model order, or you can
pass your own start_params.
The computed initial AR coefficients are n

In [ ]:
predictions =  [[]] * len(test_year_dfs)
i = 0;


for ts in test_year_dfs:
    
    if len(ts.values) == 0:
        continue
    history = [x for x in ts]
    prediction = list()
    print(len(ts))
    for t in range(len(ts.index.values)):
        output = train_year_results[i].predict(t,t, typ ="linear")
        yhat = output[0]
        prediction.append(yhat)
        obs = ts.loc[t]
        history.append(obs)
        print('predicted=%f, expected=%f' % (yhat, obs))
        error = mean_squared_error(ts, prediction)
        print('Test MSE: %.3f' % error)
        
    if i ==0:
        plt.plot(ts)
        plt.plot(prediction, color='red')
        plt.show()
        i = i + 1
        
    predictions.append(prediction)
    